# Logo Resizer


# Usage
Batch-transform sponsor logos to uniform dimensions without shape distortion.

# Features

1. 1-line code for preprocessing, trimming, resizing, and pasting logo images into the user-defined bounding box of your background template

```python
    result_img=resizer(trim(imgPrep(logo_raw)), logoBox, temp)
```

2. Preprocessing: handling images on a transparent background

For an image of `RGBA` mode, it is first pasted onto a white background of the same size as the original image, then it processed for trimming and resizing.


3. Trimming: cut out extra white background of the original image

Extra white background of the original logo image is trimmed prior to resizing to fit into the bounding box. This ensures that the logo image extends to the fullest possible size within the bounding box.

4. Resizing: adjust images that are too large or too small.

  - if logo's starting width and/or height exceed the bounding box, we will to shrink the logo so that it is just a bit smaller than the bounding box

  - if logo's starting width and/or height is smaller than the bounding box, we will to enlarge the logo so that it is just a bit larger the bounding box

5. Process multiple files and batch export to destination folder

Input files are stored in `input_PNG_JPG_logo` folder. The script reads every `*.png` and `*.JPG` file in that folder, and process them, and export the result for each image to the `result` folder. A zip file of the `result` folder is also recreated for convenience of downloading from web-hosted jupyter notebooks.

# Notes

1. Template image info:

   - size: 236x144
   - 75 dpi
   - RGB

2. Input file requirement:

   - **I implemented a check for the `*.png` and `*.jpg` filename, so that only image files with these extensions are processed.** Checking the file extension makes sure that only the correct format is included. **Modify the code in the `imgPrep` function if you want to the file to accept other formats.**


3. Coordinates for the bounding box of the logo area `[x1, y1, x2, y2]` is : [32.78, 33.48, 200.76, 112.13]. To visualize the bounding box:

    ```Python
    temp=Image.open("template.png")
    draw=ImageDraw.Draw(temp)
    draw.rectangle([32.78, 33.48, 200.76, 112.13], outline="red")
    temp.show()
    ```
    `templateWithBox.png` is same as `template.png` except that the bounding box is visible. You can also use it to test the code, such as:

    ```Python
    if __name__ =='__main__':
        temp=Image.open("tempWithBox.png")  #use `tempWithBox.png` to show the bounding box
        logo = Image.open("harbormed.png")
        logoBox=[32.78, 33.48, 200.76, 112.13]
        result=resizeLogo(trim(logo), logoBox, temp)
    ```

4. Note that after you finish running the code, you need to rename the `result` folder in the same directory. Alternatively you can save/move the `result` folder to a different directory. The reason for this is that, the code is designed to "flush" the result folder each time it runs. The old contents in the result folder will be lost and replaced with new contents.


# Examples

For the purpose of illustration, in the following examples, I marked the bounding box for the logo in GREY and the whole image frame in RED.

1. Resizing `examples/lonza-transparentBackground.png`. The raw images has a transparent background

**Before**

![lonza-raw](examples/lonza-transparentBackground.png)

**After**

![lonza-after](examples/lonza-2.png)

2. Resizing `examples/harbormed-transparentBackground.png`. The raw images has a white background

**Before**

![harbormed-raw](examples/harbormed-whiteBacground.png)

**After**

![harbormed-after](examples/harbormed-2.png)

# Code

In [1]:
from PIL import Image, ImageChops, ImageDraw
import glob
import os
import shutil

In [2]:
def trim(logo):
    """
    Trim extra whitespace surrinding the raw logo picture
    :param logo: PIL.Image object
    :return: bounding box for cropping
    """
    bg = Image.new(logo.mode, logo.size, logo.getpixel((0,0)))
    diff = ImageChops.difference(logo, bg)
    diff = ImageChops.add(diff, diff, 2, -100)
    bbox = diff.getbbox()
    if bbox:
        return logo.crop(bbox)

def resizer(logo, logoBox, template):
    """
    resizes image logos to fit into user-defined bounding box
    :param logo: PIL.Image object
    :param logoBox: Bounding box for the area. Coordinate is porvided in array format of [x1, y1, x2, y2] where [x1, y1] is the coordinate of the upper-left cornner, and [x2, y2] is that of the lower-right corner
    :param template: PIL.Image object used as a background for pasting the logo
    :return: PIL.Image object, with the resized logo pasted on the template background
    """

    #get bounding box dimensions
    x1, y1, x2, y2 = logoBox
    boxWidth=x2-x1
    boxHeight=y2-y1

    #gradually resize until image fits within the logo bounding box. First we will define
    resizeRatio=1

    #if logo width and/or height exceed the bounding box, we want to shrink the logo so that it is just a bit smaller than the bounding box
    if (logo.width>boxWidth or logo.height > boxHeight):
        while logo.width>boxWidth or logo.height > boxHeight:
            resizeRatio+=0.01
            logo=logo.resize((round(logo.width/resizeRatio), round(logo.height/resizeRatio)))

    #if logo width and/or height is smaller than the bounding box, we want to enlarge the logo so that it is just a bit larger the bounding box
    else:
        while logo.width < boxWidth and logo.height < boxHeight:
            resizeRatio-=0.01
            logo=logo.resize((round(logo.width/resizeRatio), round(logo.height/resizeRatio)))

    #paste resized logo into the  template
    deltaX=0.5*(x2-x1-logo.width)
    deltaY=0.5*(y2-y1-logo.height)

    result=template.copy()
    result.paste(logo, (round(x1+deltaX), round(y1+deltaY)))

    return result

In [3]:
def imgPrep(logo_raw):
    """
    Handles logo with transparency layer. For example, logo on a transparent background will be pasted to a white background before resizing. If image is already on a white background, nothing will be done.This step is needed before using the `trim` function
    :param logo_raw: PIL.Image object, the raw logo image
    :return: PIL.Image object, the raw logo pasted onto a white background.
    """

    #load pixel data that will be used to convert transparent background
    logo_raw.load()

    #paste the image on to a white background, in case it has a transparent background
    logo = Image.new("RGB", logo_raw.size, (255, 255, 255))

    #if the image includes a transparency layer
    if logo_raw.mode=='RGBA':
        logo.paste(logo_raw, mask = logo_raw.split()[3])
        return logo

    # if not in the RGBA mode, then nothing is done
    else:
        return logo_raw


In [4]:
def makeDir():
    """
    Handling directory of result files. Results will be stored in a folder called `result`. The program checks if such folder exsits. If it exists, its current contents will be disgarded. If it does not exist, the folder will be greated.
    :return: None
    """
    folderPath="result"
    try:
        shutil.rmtree(folderPath)  #delete 'result' folder and all its current contents if it exist
    except:
        pass  #if it doesn't exist then don't do anything
    os.mkdir(folderPath) #make a new `result` folder


In [5]:
if __name__ =='__main__':

    makeDir()

    #To test code, use templateWithBox.png with the bounding box visible
    temp=Image.open("template.png")

    #specify bounding box in the template for logo
    logoBox=[32.78, 33.48, 200.76, 112.13]

    #specify input file path and output file path
    img_path = 'input_PNG_JPG_logo/'
    output_path='result/'


    # go through every PNG/JPG file in the input_PNG_JPG_logo folder, process it, then save the output in the result folder
    # only process png and jpg files
    for filename in glob.glob(os.path.join(img_path, '*.png'))+glob.glob(os.path.join(img_path, '*.PNG'))+glob.glob(os.path.join(img_path, '*.jpg'))+glob.glob(os.path.join(img_path, '*.JPG')):
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            fn=(filename.split('/')[-1]).split('.')[0] #get the name of the image. E.g. "harbormed" instead of "input_PNG_JPG_Logo/harbormed.png"
            new_filename=output_path+fn+"_2.png"

            #load raw logo
            logo_raw = Image.open(filename)

            #preprocess, trim,  resize the logo, then paste it onto the template
            result_img=resizer(trim(imgPrep(logo_raw)), logoBox, temp)
            result_img.save(new_filename)

    shutil.make_archive("result", 'zip', "result") #zip up file for easy download in web-based Jupyter


